In [34]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [28]:
url = 'https://fbref.com/en/comps/24/stats/Serie-A-Stats'

In [29]:
data = requests.get(url)

In [30]:
data.text

'    \n      \n<!DOCTYPE html>\n<html data-version="klecko-" data-root="/home/fb/deploy/www/base" lang="en" class="no-js" >\n<head>\n    <meta charset="utf-8">\n    <meta http-equiv="x-ua-compatible" content="ie=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=2.0" />\n    <link rel="dns-prefetch" href="https://cdn.ssref.net/req/202311303" />\n<!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->\n<script type="text/javascript" async=true>\n    (function() {\n\tvar host = window.location.hostname;\n\tvar element = document.createElement(\'script\');\n\tvar firstScript = document.getElementsByTagName(\'script\')[0];\n\tvar url = \'https://cmp.quantcast.com\'\n\t    .concat(\'/choice/\', \'XwNYEpNeFfhfr\', \'/\', host, \n\t\t    \'/choice.js?tag_version=V2\');\n\tvar uspTries = 0;\n\tvar uspTriesLimit = 3;\n\telement.async = true;\n\telement.type = \'text/javascript\';\n\telement.src = url;\n\t\n\tfirstScript.parentNode.insertBef

In [32]:
soup = BeautifulSoup(data.text)

In [39]:
standings_table = soup.select('table.stats_table')[2]

IndexError: list index out of range

In [38]:
standings_table

<table class="stats_table sortable min_width" data-cols-to-freeze=",1" id="stats_squads_standard_against"> <caption>Squad Standard Stats <span style="color: #666; font-size:smaller">2023 Série A</span> Table</caption> <colgroup><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/></colgroup> <thead> <tr class="over_header"> <th aria-label="" class="over_header center" colspan="4" data-stat=""></th> <th aria-label="" class="over_header center group_start" colspan="4" data-stat="header_playing">Playing Time</th> <th aria-label="" class="over_header center group_start" colspan="8" data-stat="header_performance">Performance</th> <th aria-label="" class="over_header center group_start" colspan="4" data-stat="header_expected">Expected</th> <th aria-label="" class="over_header center group_start" colspan="2" data-stat="header_progression">Progression</th> 